<DIV ALIGN=CENTER>

# Introduction to Text Mining
## Professor Robert J. Brunner
  
</DIV>  
-----
-----


## Introduction

In this IPython Notebook, we explore more advanced machine learning
techniques with text data. First, we introduce the concept of n-grams,
which are combinations of one or more tokens. For example, bigrams are
combinations of two tokens, while trigrams are combinations of three
tokens. Next, we introduce the concept of stemming, which is used to
convert tokens into their root forms so that token frequencies match the
use of the root token rather than being spread across multiple similar
tokens. Finally, we explore the application of clustering and feature
selection to text data.

-----

In [1]:
# Set up Notebook

% matplotlib inline

# Standard imports
import numpy as np

/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/conda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## n-grams

Formally, a [_n-gram_][ngd] is a contiguous sequence of **n** items from a
parent sequence of items, such as characters or words in a text
document. In general, we will focus solely on words in a document. Thus,
our initial approach has simply been to look at unigrams or single
words in a document when building a classification model. However,
sometimes the combination of words can be more descriptive, for example,
_unbelievably bad_ is generally viewed as a more powerful description
than just _bad_. As a result, the concept of an _n-gram_ was created,
where collections of words can be treated as features. In fact google
allows a user to search for [specific n-gram][gnv] combinations in books that
they have scanned.

While this clearly can improve classification power, it also increases
computational requirements. This is a result of the exponential rise in
the number of possible features. For example, given $n$ words, we have
$n \times (n - 1)$ possible bigrams, and so on for higher order
combinations. While this is not a problem for small vocabularies, for
larger vocabularies (and corresponding documents) the number of possible
features can quickly become very large. Thus, many text mining
applications will make use of Hadoop or Spark clusters to leverage the
inherent parallelism in these tasks.

To demonstrate using n-grams, we first demonstrate the concept on a
single sentence.

-----
[gnv]: https://books.google.com/ngrams
[ngd]: https://en.wikipedia.org/wiki/N-gram

In [2]:
my_text = 'INFO490 introduces many concepts in data science.'

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1,3))

tk_func = cv.build_analyzer()

import pprint
pp = pprint.PrettyPrinter(indent=2, depth=1, width=80, compact=True)

pp.pprint(tk_func(my_text))

[ 'info490', 'introduces', 'many', 'concepts', 'in', 'data', 'science',
  'info490 introduces', 'introduces many', 'many concepts', 'concepts in',
  'in data', 'data science', 'info490 introduces many',
  'introduces many concepts', 'many concepts in', 'concepts in data',
  'in data science']


In [3]:
in_list = []
in_list.append(my_text)

cv = cv.fit(in_list)

import operator
my_voc = sorted(cv.vocabulary_.items(), key=operator.itemgetter(1))

print('Token mapping:')
print(40*'-')

for tokens, rank in my_voc:
    print(rank, tokens)

print(40*'-')
out_list = ['INFO490 is data science']
xsm = cv.transform(out_list)
print(out_list)
print(40*'-')
print(xsm.todense())

Token mapping:
----------------------------------------
0 concepts
1 concepts in
2 concepts in data
3 data
4 data science
5 in
6 in data
7 in data science
8 info490
9 info490 introduces
10 info490 introduces many
11 introduces
12 introduces many
13 introduces many concepts
14 many
15 many concepts
16 many concepts in
17 science
----------------------------------------
['INFO490 is data science']
----------------------------------------
[[0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1]]


-----

### Student Activity

In the preceding cells, we used `CountVectorizer` to create n-gram
tokens. Now that you have run the Notebook, go back and make the
following changes to see how the results change.

1. Change the `CountVectorizer` to use stop words and change the tokens
to all lowercase. How does this change the tokens and mappings? 
2. Create your own sentence, do the tokens and n-grams make sense?
3. Try changing the ngram range to different values (e.g.,
`ngram_range=(1,4)` or `ngram_range=(2,3)`). Notice how the number of
tokens quickly increase.


-----

## N-gram Classification

Having n-grams offers improved classification, since word or token
combinations often include more information than single words or tokens.
For example, _University Illinois_ means more than just _University_ and
_Illinois_. We can build on our previous simple text classification
pipeline to now develop a more complete code example that builds a
feature vector containing both single words and b-grams from the
documents. We use this new sparse matrix to classify the documents by
using our simple Naive Bayes classifier, which obtains slightly better
results. First, we load the movie review data.

-----

In [4]:
import nltk
mvr = nltk.corpus.movie_reviews

from sklearn.datasets import load_files

data_dir = '/home/data_scientist/data/nltk_data/corpora/movie_reviews'


#data_dir = '/home/data_scientist/nltk_data/corpora/movie_reviews'
mvr = load_files(data_dir, shuffle = False)
print('Number of Reviews: {0}'.format(len(mvr.data)))

from sklearn.cross_validation import train_test_split

mvr_train, mvr_test, y_train, y_test = train_test_split(
    mvr.data, mvr.target, test_size=0.25, random_state=23)

FileNotFoundError: [Errno 2] No such file or directory: '/home/data_scientist/data/nltk_data/corpora/movie_reviews'

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics

tools = [('cv', CountVectorizer()), ('nb', MultinomialNB())]
pclf = Pipeline(tools)


# Lowercase and restrict ourselves to about half the available features
pclf.set_params(cv__stop_words = 'english', \
                cv__ngram_range=(1,2), \
                cv__lowercase=True)

pclf.fit(mvr_train, y_train)
y_pred = pclf.predict(mvr_test)
print(metrics.classification_report(y_test, y_pred, target_names = mvr.target_names))

NameError: name 'mvr_train' is not defined

In [27]:
# Extract the classifier
clf = pclf.steps[1][1]
print('Number of Features = {}'.format(clf.feature_log_prob_.shape[1]))

Number of Features = 421010


-----

Now we can repeat the results, but now use unigrams, bigrams, and
trigrams. Since this will produce a document term matrix that likely
exceeds the computational resources of our Docker containers, we impose
two cuts on the tokens included in our DTM. First, we impose a minimum
feature term that requires a term to be present in at least two
documents. Second, we set a maximum frequency of 50%, such that any term
occurring in more than fifty percent of all documents will be ignored
(these are likely all stop words, but this value can be reduced to a
lower value to trim frequently occurring words that add little to the
accuracy. Notice that our overall number of features is much smaller,
even though we are also using trigrams in this classification pipeline.

-----

In [28]:
pclf.set_params(cv__stop_words = 'english', \
                cv__ngram_range=(1,3), \
                cv__lowercase=True, \
                cv__min_df=2, \
                cv__max_df=0.5)

pclf.fit(mvr_train, y_train)
y_pred = pclf.predict(mvr_test)
print(metrics.classification_report(y_test, y_pred, target_names = mvr.target_names))

             precision    recall  f1-score   support

        neg       0.84      0.78      0.81       259
        pos       0.78      0.83      0.81       241

avg / total       0.81      0.81      0.81       500



In [29]:
# Extract the classifier
clf = pclf.steps[1][1]
print('Number of Features = {}'.format(clf.feature_log_prob_.shape[1]))

Number of Features = 62735


-----

### Student Activity

In the preceding cells, we included n-grams in the classification
process by using a `CountVectorizer`.  Now that you have run the
Notebook, go back and make the following changes to see how the results
change.

1. Try to determine a value of `max_df` that replicates the effects of
using stop words. 
2. Change `CountVectorizer` to `TfidfVectorizer`. Do the results change?
3. Try using a more powerful classification algorithm, as opposed to
`MultinomialNB`. Do the results change?

-----

## Stemming

So far, we have looked at several techniques to remove redundant or
unimportant features. For example, we changed the case of all text to
lowercase and we have applied stop words. However, there still is the
issue of different forms of the same word, for example compute,
computer, computed, and computing. The process of changing words back to
their root, or basic form (by removing prefixes and suffixes) is known as
stemming. 

The most widely used stemmer, or program/method that performs stemming,
is the _Porter Stemmer_, which was originally published in 1980 by
Martin Porter. An improved version was released in 2000, which fixed a
number of errors. NLTK includes the Porter Stemmer, which can be used
with scikit learn by creating a special function that tokenizes text
documents and passing this function as an argument to the
`CountVectorizer` via the `tokenizer` attribute. By performing stemming
inside this tokenize method, we can return a set of tokens for a
document that have been stemmed. In the following code cell, we use a
custom `tokenize` method that first builds a list of tokens by using
nltk, and then maps the Porter stemmer to the list of tokens to generate
a stemmed list.


-----
[ws]: https://en.wikipedia.org/wiki/Stemming

In [30]:
import string
import nltk
from nltk.stem.porter import PorterStemmer

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in string.punctuation]

    stemmer = PorterStemmer()
    stems = map(stemmer.stem, tokens)
    return stems

pclf.set_params(cv__stop_words = 'english', \
                cv__ngram_range=(1,3), \
                cv__lowercase=True, \
                cv__tokenizer=tokenize)

pclf.fit(mvr_train, y_train)
y_pred = pclf.predict(mvr_test)
print(metrics.classification_report(y_test, y_pred, target_names = mvr.target_names))

             precision    recall  f1-score   support

        neg       0.83      0.77      0.80       259
        pos       0.77      0.83      0.80       241

avg / total       0.80      0.80      0.80       500



In [31]:
# Extract the classifier
clf = pclf.steps[1][1]
print('Number of Features = {}'.format(clf.feature_log_prob_.shape[1]))

Number of Features = 80529


-----

### Student Activity

In the preceding cells, we incorporated the Porter Stemmer into the
classification pipeline. Now that you have run the Notebook, go back and
make the following changes to see how the results change.

1. Did the Porter Stemmer improve the classification results (note you
need to be sure you are comparing exactly the same pipeline (including
the use of ngrams)? 

2. Can you compute the number of features that the Porter Stemmer
generates? How does this compare the number of features without stemming?

3. Try using a different stemming algorithm, such as [_snowball_][nsw].
How do the classification results change? 

-----
[nsw]: http://www.nltk.org/api/nltk.stem.html#module-nltk.stem.snowball

## Clustering Analysis

We can also apply clustering analysis to our feature matrix. While
finding an unknown number of clusters in text documents can be
difficult, we can learn about our data by identifying the clusters for
our **known** labels. To demonstrate, in the following code cells, we
employ k-means to find two clusters in our feature matrix (the moview reviews), after
which we identify the most frequently used words in each cluster.

-----

In [32]:
from sklearn.cluster import KMeans

true_k = 2

km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

from sklearn.feature_extraction.text import CountVectorizer

# Verify attributes

cv = CountVectorizer(stop_words = 'english', \
                     ngram_range=(1, 3), max_features=100000)

train_counts = cv.fit_transform(mvr_train)
test_data = cv.transform(mvr_test)

km.fit(test_data)

KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=2, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [33]:
top_tokens = 20
labels = ['Neg', 'Pos']

print('Top {} tokens per cluster:\n'.format(top_tokens))

order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = cv.get_feature_names()

for idx in range(true_k):
    print("Cluster {0}:".format(idx), end='')
    for jdx in order_centroids[idx, :top_tokens]:
        print(' {0}'.format(terms[jdx]), end='')
    print('\n')

Top 20 tokens per cluster:

Cluster 0: film like movie just time good character story way characters scene does really films make life plot people man scenes

Cluster 1: movie film like just good time story character way make new plot characters little bad man movies people does really



-----

We can perform the same analysis on a more complex problem by analyzing
the twenty newsgroup data set. First we load the data, after which we
apply k-means and identify the most common tokens in each cluster.

-----

In [34]:
# load dataset
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(data_home='/home/data_scientist/data/textdm', subset='train', shuffle=True, random_state=23)
test = fetch_20newsgroups(data_home='/home/data_scientist/data/textdm', subset='test', shuffle=True, random_state=23)

In [35]:
from sklearn.cluster import KMeans

true_k = 20

km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)

from sklearn.feature_extraction.text import CountVectorizer

# Verify attributes

cv = CountVectorizer(stop_words = 'english', max_features=100000)
train_counts = cv.fit_transform(train['data'])
test_data = cv.transform(test['data'])

km.fit(test_data)

KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=20, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [36]:
top_tokens = 20
labels = test['target']

print('Top {} tokens per cluster:\n'.format(top_tokens))

order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = cv.get_feature_names()

for idx in range(true_k):
    print("Cluster {0}:".format(idx), end='')
    for jdx in order_centroids[idx, :top_tokens]:
        print(' {0}'.format(terms[jdx]), end='')
    print('\n')

Top 20 tokens per cluster:

Cluster 0: cancer hiv health 1993 medical number april 25 11 disease drug information volume newsletter hicnet page pages patients national aids

Cluster 1: edu subject com lines organization writes article university don like just posting host nntp know people think does time new

Cluster 2: dos windows microsoft tcp ms mouse amiga software pc graphics higher macintosh network mbytes version 00 ip memory support card

Cluster 3: 92 12 10 hiv 17 11 aids patients et 03 30 medical 25 milk cd4 31 tb 1993 number 04

Cluster 4: jpeg image gif file color format images quality version files bit free programs available use jfif software don display edu

Cluster 5: said people president adl think mr know don god openwindows did stephanopoulos just general time use files going mac say

Cluster 6: edu 25 com file image ftp use server gopher images pub azerbaijan windows program kinsey information time files make like

Cluster 7: jehovah elohim lord god christ father mc

-----

### Student Activity

In the preceding cells, we used k-means clustering to find clusters in
text data, and to identify the most important tokens in these clusters.
Now that you have run the Notebook, go back and make the following
changes to see how the results change.

1. Change the vectorizer to use TF-IDF. Does this change the tokens in
each cluster? 
2. Change the vectorizer to use ngrams. Does this change the tokens in
each cluster?
3. Include stemming in the vectorizer. Does this change the tokens in
each cluster?
4. Try using DBSCAN instead of k-means. How many clusters are found?
What are the tokens in each cluster?

Finally, the k-means algorithm finds clusters. Do these clusters map
directly into the _real_ categories? Feel free to discuss this on the
course forums.

-----

## Dimension Reduction

The document term matrices we have constructed in these examples can
become quite large. We have already reduced the number of features used
in classification problems by using stop words, by using a consistent
case, and by performing stemming. On the other hand, we have enabled
exponential increases in the feature space by including n-grams
(although we once again restricted the feature space by using the
`max_features` or the `max_df` and `min_df` attributes). The
traditional dimensional reduction technique we have used in the past is
PCA. However, PCA can be difficult to use with text data given the large
sizes of the matrices (since a matrix inversion can be required). We can
employ alternative techniques, such as incremental PCA or Truncated SVD. 

But in reality, we are less interested in finding a reduced dimensional
space than we are in removing features that contain little or no
information (combining features is essentially increasing the ngram
range). In this case, the problem of dimension reduction becomes one of
optimal feature selection. For this, we can use the scikit learn
`SelectKBest` method to identify the best $k$ features. In the following
code cells, we first create a vectorizer, train it on data to
demonstrate the accuracy and the number of features required to achieve
that accuracy. After which, we employ `SelectKBest` to identify the best
number of features, where number is predetermined, to achieve a similar
accuracy. In the end, we find that less than ten percent of the original
features are required to achieve the same level of accuracy.

-----

In [37]:
# Following Example was insipred by scikit learn demo
# http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html

from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')

In [38]:
# First, train on normal set of features, baseline performance.

train_counts = tf.fit_transform(train['data'])
test_data = tf.transform(test['data'])

nb = MultinomialNB()
nb = nb.fit(train_counts, train['target'])
predicted = nb.predict(test_data)

print("Prediction accuracy = {0:5.1f}%".format(100.0 * nb.score(test_data, test['target'])))
print('Number of Features = {}'.format(nb.feature_log_prob_.shape[1]))

Prediction accuracy =  82.0%
Number of Features = 129792


In [39]:
# Now employ feature selection
from sklearn.feature_selection import SelectKBest, chi2

# Number of features to keep
num_k = 10000

# Employ select k best wiht chi2 since this works with sparse matrices.
ch2 = SelectKBest(chi2, k=num_k)
xtr = ch2.fit_transform(train_counts, train['target'])
xt = ch2.transform(test_data)

In [40]:
# Train simple model and compute accuracy
nb = nb.fit(xtr, train['target'])
predicted = nb.predict(xt)

print("NB prediction accuracy = {0:5.1f}%".format(100.0 * nb.score(xt, test['target'])))
print('Number of Features = {}'.format(nb.feature_log_prob_.shape[1]))

NB prediction accuracy =  82.0%
Number of Features = 10000


-----

We can use the feature selection to identify the top features for each
category. First we extract the feature names, after which we extract the
importance (or support) for each feature. By sorting the array containing
the important features, we can identify the top tokens.

-----

In [41]:
feature_names = tf.get_feature_names()

indices = ch2.get_support(indices=True)
feature_names = np.array([feature_names[idx] for idx in indices])

In [42]:
import pprint
pp = pprint.PrettyPrinter(indent=2, depth=1, width=80, compact=True)

top_count = 5

for idx, target in enumerate(train['target_names']):
    top_names = np.argsort(nb.coef_[idx])[-top_count:]
    tn_lst = [name for name in feature_names[top_names]]
    tn_lst.reverse()

    print('\n{0}:'.format(target))
    pp.pprint(tn_lst)


alt.atheism:
['keith', 'god', 'caltech', 'atheists', 'livesey']

comp.graphics:
['graphics', 'image', 'thanks', '3d', 'files']

comp.os.ms-windows.misc:
['windows', 'dos', 'file', 'files', 'driver']

comp.sys.ibm.pc.hardware:
['drive', 'card', 'scsi', 'ide', 'bus']

comp.sys.mac.hardware:
['mac', 'apple', 'quadra', 'drive', 'centris']

comp.windows.x:
['window', 'motif', 'mit', 'com', 'server']

misc.forsale:
['sale', 'offer', 'shipping', 'distribution', 'condition']

rec.autos:
['car', 'cars', 'com', 'article', 'engine']

rec.motorcycles:
['bike', 'dod', 'com', 'ride', 'motorcycle']

rec.sport.baseball:
['baseball', 'year', 'game', 'team', 'players']

rec.sport.hockey:
['hockey', 'team', 'game', 'ca', 'nhl']

sci.crypt:
['clipper', 'key', 'encryption', 'chip', 'com']

sci.electronics:
['com', 'use', 'circuit', 'host', 'power']

sci.med:
['pitt', 'geb', 'banks', 'gordon', 'com']

sci.space:
['space', 'nasa', 'moon', 'orbit', 'henry']

soc.religion.christian:
['god', 'jesus', 'christia

-----

### Student Activity

In the preceding cells, we used feature selection to identify the most
important features in our simple classification pipeline. Now that you
have run the Notebook, go back and make the following changes to see how
the results change.

1. Change the vectorizer to change the case of all words an to employ
stemming. How do the results (tokens) change?

2. Change the classification algorithm to a more accurate method. How do
the results change? How does the computational time change?

Finally, what do the list of tokens say about the fact we did not remove
headers or footers from the newsgroup postings? Feel free to comment on
these questions in the course forum.

-----